In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame as DF
from sympy import diff
from sympy.abc import x,y
from sympy import cos
from sympy import *
from sympy import lambdify
from itertools import product
import random
from collections import defaultdict
import copy
import time

**Define path**

In [2]:
# Defining path to folder raw data is saved in
path = "C:/Users/robtk/OneDrive/Desktop/DIAS Internship/Raw data/Harmonic oscillator warm up/"

### Set display of pandas dataframes and Series to 11 decimal places

In [3]:
pd.set_option("display.precision", 11)

**Define the commutator**

In [4]:
def commutator(A,B):
    
    # matmul() multiplies two matrices together
    return np.matmul(A,B) - np.matmul(B,A)

### Define Lagrangian

In [15]:
def L(X_list, V_list, g):
    
    X_array = np.array(X_list)
    V_array = np.array(X_list)
    term_1 =  np.sum( np.trace( (V_array)**2 ) )
    
    # Calculate the sum of commutators found in the second term of the lagrangian
    for el in X_list:
        X_cut = [x for x in X_list if x != el]
        Sum2 = np.sum( np.trace( (commutator(el,X_cut))**2 ) ) 
    
    term_2 = 1/2 * Sum2
    
    return 1/(2*g**2) * (term_1 + term_2)

**Define the Hamiltonian**

In [17]:
def H(X_list, V_list, g):
    
    V_array = np.array(X_list)
    V_array_T = np.transpose(V_array)
    
    return sum( V_array_T * V_array /g**2) - L(X_list, V_list, g)
    

Initial reference point is x_0 = (X, V)

**Define the acceleration equation of motion as gotten from the research paper.**

In [18]:
def acceleration(X_list):
    
    # This function creates a list of all the accelerations of all the coordinates from a list of the coordinate matrices.
    
    # List to be filled with accelerations for the coordinates
    A_list = []
    
    # Turn the coordinates into an array so all X^j can put into the equation of motion in one line.
    array = np.array(X_list)
    
    # for loop to calculate all the coordinate accelerations.
    for el in X_list:
        
        # Doing the equation of motion
        # el == X^i and array == X^j in the equation of motion
        equation_of_motion = commutator(array ,commutator(el, array))
        
        # sum() carries out the summation symbol in the equation of motion and so the acceleration is calculated.
        # append the acceleration to the list.
        A_list.append(sum(equation_of_motion))
        
    return A_list

**Code for evolving the initial point in phase space for 1000 seconds so that the system is thermalised.**

In [46]:
def thermalising_algorithm(X_list, V_list, A_list, delta_t):
    
    # Define initial conditions as numpy arrays
    X_array_old = np.array(X_list)
    V_array_old = np.array(V_list)
    A_array_old = np.array(A_list)
    
    i = 0
    # Repeating algorithm for 1000 seconds to thermalise the system.
    
    while i < 100/delta_t:
        
        # b) velocity Verlet 1 to get new positions from old positions, momentums and rate of change of momentums
        
        X_array_new = (X_array_old + V_array_old * delta_t + 1/2 * A_array_old * delta_t**2)

        
        # c) Use equations of motion to get new acceleration values
        
        A_array_new = np.array(acceleration(X_array_new.tolist()))

        # d) Use Velocity Verlet 2 to get new momentums

        V_array_new = (V_array_old + (1/2) * (A_array_new + A_array_old) * delta_t)
                
        # Update coordinates
        X_array_old = X_array_new
        V_array_old = V_array_new
        A_array_old = A_array_new
        
        if i % 10000000 == True:
            print("Thermal", i, X_array_new)

        
        i += 1
        
    return X_array_new, V_array_new, A_array_new

**Code for evolving the simulation and recording the position and momentum after it was thermalised.**

In [42]:
def evolving_algorithm(X_list, V_list, A_list, delta_t, record_steps, simulation_repetitions, 
                       X_labels, V_labels, N, times):
    
    i = 1
    
    # Define initial conditions as numpy arrays
    X_array_old = np.array(X_list)
    V_array_old = np.array(V_list)
    A_array_old = np.array(A_list)
    
    # Coordinates dictionaries
    recorded_X1 = { "0" : X_list[0].tolist()}
    recorded_X2 = { "0" : X_list[1].tolist()}
    recorded_V1 = { "0" : V_list[0].tolist()}
    recorded_V2 = { "0" : V_list[1].tolist()}



    # Repeating algorithm
    while i < simulation_repetitions:
        
        # b) velocity Verlet 1 to get new positions from old positions, momentums and rate of change of momentums
        
        X_array_new = (X_array_old + V_array_old * delta_t + 1/2 * A_array_old * delta_t**2)

        
        # c) Use equations of motion to get new acceleration values
        
        A_array_new = np.array(acceleration(X_array_new.tolist()))


        # d) Use Velocity Verlet 2 to get new momentums

        V_array_new = (V_array_old + (1/2) * (A_array_new + A_array_old) * delta_t)
        
        
        # Recording position, momentum, and rate of change of momentum every 10th step
        if i % record_steps == 0:

            recorded_X1[f"{delta_t*record_steps*i}"] = X_array_new[0].tolist()
            recorded_X2[f"{delta_t*record_steps*i}"] = X_array_new[1].tolist()
            recorded_V1[f"{delta_t*record_steps*i}"] = V_array_new[0].tolist()
            recorded_V2[f"{delta_t*record_steps*i}"] = V_array_new[1].tolist()
            
            print(X_array_new)

        
            
        # Update coordinates
        X_array_old = X_array_new
        V_array_old = V_array_new
        A_array_old = A_array_new
         

        i += 1
    
    return recorded_X1, recorded_X2, recorded_V1, recorded_V2

**Create initial position matrices: Take X^i to have random (Gaussian) matrix elements, and be traceless.**

In [51]:
def simulation(dim, N, delta_t):
    # N is the length of the side of the individual position square matrices.
    # dim is the number of position coordinates we need.
    
    # Create position coordinate matrices with gaussian distributed elements.
    
    X_list = np.random.normal(0, 1, size = (dim, N, N))
    
    print(X_list)
    # The velocity Verlet algorithm conserves Gauss' law, so if it starts off true, it remains true. 
    # So just ensure the initial V^i are zero matrices.  

    V_initial = np.zeros((N, N))
    i = 0
    V_list = []
    while i < len(X_list):
        V_list.append(V_initial)
        i += 1
    
    # Use the scalar equation of motion in our gauge to calculate the initial accelerations from the initial positions
    
    A_list = acceleration(X_list)
    

    # Define number of steps between measurement recordings
    record_steps = 100
    
    # For checking if H is conserved
    #print(H())

    
    # How many iterations the simulation will go on for after it has been thermalised.
    simulation_repetitions = 1000
    # Thermalise the system. , i.e. evolve it for long enough time that it presumably reaches a "typical configuration".
    # Here Swapno suggests to evolve it, starting anywhere, for 1000 seconds, dt = 10e-5 s.
 
    thermalised_X_array, thermalised_V_array, thermalised_A_array = thermalising_algorithm(X_list, V_list, A_list, delta_t)
        
    # Labels for coordinates dictionary
    X_labels = []
    j = 1
    while j <= dim:
        X_labels.append(f"X {j}")
        
        j+=1
    
    V_labels = []
    j = 1
    while j <= dim:
        V_labels.append(f"V {j}")
        
        j+=1
    
    
    # Define the times after thermalisation that the coordinates were recorded at.
    times = np.arange(0, simulation_repetitions*delta_t + delta_t*record_steps, delta_t*record_steps)
    
    # Now that the system is thermalised and the thermalised initial conditions are found, evolve the system
    # and run the data collecting simulation.
    recorded_X1, recorded_X2, recorded_V1, recorded_V2 = evolving_algorithm(thermalised_X_array,
                                                                            thermalised_V_array, 
                                                                            thermalised_A_array,
                                          delta_t, record_steps, simulation_repetitions, 
                                          X_labels, V_labels, N, times)
    
    # Make dataframes out of the dictionaries
    
    # Check if H is conserved

    X1 = pd.Series(recorded_X1).T
    X2 = pd.Series(recorded_X2).T
    V1 = pd.Series(recorded_V1).T
    V2 = pd.Series(recorded_V2).T

    X = pd.concat([X1, X2], axis = 1)
    X = X.rename(columns = {0:"X 1", 1:"X 2"})
    
    V = pd.concat([V1, V2], axis = 1)
    V = V.rename(columns = {0:"V 1", 1:"V 2"})
    

    
    return X, V

In [52]:
start = time.time()
X, V= simulation(2, 2, 10**-3)

print(time.time()-start)

[[[-0.64921    -1.89889138]
  [ 1.11378162  1.35621667]]

 [[ 0.13419785  0.69137344]
  [ 1.73933915  0.15185043]]]
Thermal 1 [[[-0.64921    -1.89889138]
  [ 1.11378162  1.35621667]]

 [[ 0.13419785  0.69137344]
  [ 1.73933915  0.15185043]]]
[[[-25.63833653 -91.76651652]
  [-75.04556035  26.3453432 ]]

 [[ 59.18100864 208.7552667 ]
  [171.23160059 -58.89496037]]]
[[[-25.63774838 -91.7665779 ]
  [-75.04908416  26.34475504]]

 [[ 59.18265096 208.7601326 ]
  [171.23406488 -58.89660268]]]
[[[-25.63705026 -91.76646504]
  [-75.05281309  26.34405693]]

 [[ 59.18434157 208.76507508]
  [171.23643919 -58.89829329]]]
[[[-25.63624445 -91.76618149]
  [-75.05674293  26.34325112]]

 [[ 59.18607949 208.77009257]
  [171.23872534 -58.90003121]]]
[[[-25.6353334  -91.76573116]
  [-75.06086911  26.34234007]]

 [[ 59.18786363 208.77518337]
  [171.24092536 -58.90181536]]]
[[[-25.63431975 -91.76511826]
  [-75.06518665  26.34132642]]

 [[ 59.18969284 208.78034561]
  [171.24304141 -58.90364456]]]
[[[-25.6332063

In [54]:
X

,X 1,X 2
0,"[[-25.638812687739993, -91.76627763573693], [-...","[[59.17941552987677, 208.75047881451303], [171..."
0.009999999999999998,"[[-25.638336533779736, -91.76651651917533], [-...","[[59.18100864431433, 208.75526669847144], [171..."
0.019999999999999997,"[[-25.637748375612812, -91.7665779036196], [-7...","[[59.18265095740752, 208.76013259606927], [171..."
0.03,"[[-25.637050262750538, -91.76646503673459], [-...","[[59.18434156846955, 208.7650750784062], [171...."
0.039999999999999994,"[[-25.636244450645783, -91.76618149253375], [-...","[[59.186079486328325, 208.77009257306216], [17..."
0.049999999999999996,"[[-25.635333396497664, -91.76573116473581], [-...","[[59.187863631175, 208.7751833670365], [171.24..."
0.06,"[[-25.63431975467819, -91.76511825952142], [-7...","[[59.18969283657943, 208.78034560995542], [171..."
0.06999999999999999,"[[-25.63320637178904, -91.76434728770421], [-7...","[[59.19156585166818, 208.7855773175364], [171...."
0.07999999999999999,"[[-25.631996281358198, -91.76342305633099], [-...","[[59.19348134346092, 208.79087637530415], [171..."
0.09,"[[-25.630692698185307, -91.76235065972607], [-...","[[59.195437899361714, 208.79624054255325], [17..."


### Set display of pandas dataframes and Series to 11 decimal places

In [2]:
pd.set_option("display.precision", 11)

In [55]:
X.to_pickle(path+"Xb.pkl")
V.to_pickle(path+"Vb.pkl")

# Lyapunov exponent

1. Thermalise system
1. Perturb system
1. Evolve both original and perturbed system, measuring the exponential rate at which they diverge.

In [ ]:
def simulation_2_Points(dim, N, delta_t):
    # N is the length of the side of the individual position square matrices.
    # dim is the number of position coordinates we need.
    
    # Create position coordinate matrices with gaussian distributed elements.
    
    X_list = np.random.normal(0, 1, size = (dim, N, N))
    
    # The velocity Verlet algorithm conserves Gauss' law, so if it starts off true, it remains true. 
    # So just ensure the initial V^i are zero matrices.  

    V_initial = np.zeros((N, N))
    i = 0
    V_list = []
    while i < len(X_list):
        V_list.append(V_initial)
        i += 1
    
    # Use the scalar equation of motion in our gauge to calculate the initial accelerations from the initial positions
    
    A_list = acceleration(X_list)

    # Define number of steps between measurement recordings
    record_steps = 100000
    
    # How many iterations the simulation will go on for after it has been thermalised.
    simulation_repetitions = 10000000
    
    
    # Thermalise the system. , i.e. evolve it for long enough time that it presumably reaches a "typical configuration".
    # Here Swapno suggests to evolve it, starting anywhere, for 1000 seconds, dt = 10e-5 s.
    thermalised_X_array, thermalised_V_array, thermalised_A_array = thermalising_algorithm(X_list, V_list, A_list, delta_t)
        
    
    #### New Code ####
    
    # Perturb the thermalised_initial coordinates to get the second system
    
    
    
        
    ####          #### 
    
    # Labels for coordinates dictionary
    X_labels = []
    j = 1
    while j <= dim:
        X_labels.append(f"X {j}")
        
        j+=1
    
    V_labels = []
    j = 1
    while j <= dim:
        V_labels.append(f"V {j}")
        
        j+=1
    
    
    # Define the times after thermalisation that the coordinates were recorded at.
    times = np.arange(0, simulation_repetitions*delta_t + delta_t*record_steps, delta_t*record_steps)
    
    # Make dataframes out of the dictionaries
    
    # Check if H is conserved

    X1 = pd.Series(recorded_X1).T
    X2 = pd.Series(recorded_X2).T
    V1 = pd.Series(recorded_V1).T
    V2 = pd.Series(recorded_V2).T

    X = pd.concat([X1, X2], axis = 1)
    X = X.rename(columns = {0:"X 1", 1:"X 2"})
    
    V = pd.concat([V1, V2], axis = 1)
    V = V.rename(columns = {0:"V 1", 1:"V 2"})
    

    
    return X, V